In [1]:
from pathlib import Path
import json

# 读取 selection.json
PROJECT_ROOT = Path("/Users/apple/Desktop/VirtualFurnishing")
selection_path = PROJECT_ROOT / "furniture_select" / "selection.json"

with selection_path.open("r", encoding="utf-8") as f:
    selection = json.load(f)

# 简要查看：条目数量与前几个元素
print(f"Loaded {len(selection)} items from selection.json")
for item in selection[:3]:
    print(item)


Loaded 5 items from selection.json
{'model_id': 'b8b746a5-bf6e-4679-a702-e783140cc4d8', 'super-category': 'Sofa', 'category': 'L-shaped Sofa', 'style': 'Modern', 'price_cny': 4660, 'xLen': 2.087, 'zLen': 0.961, 'footprint_m2': 2.005607}
{'model_id': '06791438-cfa8-4e29-95cc-4ff8107e3456', 'super-category': 'Cabinet/Shelf/Desk', 'category': 'Coffee Table', 'style': 'Modern', 'price_cny': 200, 'xLen': 1.08, 'zLen': 0.174, 'footprint_m2': 0.18792}
{'model_id': '1a4af735-398a-483b-ad94-68baeb0517bd', 'super-category': 'Cabinet/Shelf/Desk', 'category': 'TV Stand', 'style': 'Modern', 'price_cny': 280, 'xLen': 1.4, 'zLen': 0.482, 'footprint_m2': 0.6748}


In [3]:
import sys
import platform
import numpy as np

print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"NumPy: {np.__version__}")

# NumPy 2.x 与部分已编译模块（如 onnxruntime<2，部分扩展）不兼容
# 若为 2.x，则给出明确修复提示并中断，避免后续单元格崩溃
if not np.__version__.startswith("1."):
    raise RuntimeError(
        "检测到 NumPy>=2（当前版本为 %s）。请将 NumPy 降级到 <2 并确保 onnxruntime/rembg 兼容。" % np.__version__
        + "\n修复步骤：\n"
        + "1) 在当前虚拟环境中执行：\n"
        + "   pip install --upgrade 'numpy<2' 'onnxruntime>=1.16,<2' rembg\n"
        + "2) 重启 Jupyter 内核后再运行。"
    )


Python: 3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]
Platform: macOS-13.0-arm64-arm-64bit
NumPy: 1.26.4


In [9]:
from pathlib import Path
import sys
import io
import cv2
import numpy as np

# 依赖检查：需要 rembg 才能高质量去背景
try:
    from rembg import remove  # 使用预训练人像/通用抠图模型
except Exception as e:
    raise ImportError(
        "需要安装 rembg 库以去除背景，请先安装：pip install rembg"
    ) from e

PROJECT_ROOT = Path("/Users/apple/Desktop/VirtualFurnishing")
images_root = PROJECT_ROOT / "data" / "modern_images"
rgba_out_dir = PROJECT_ROOT / "outputs" / "rgba"
rgba_out_dir.mkdir(parents=True, exist_ok=True)

# 允许的图片扩展名
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}

def find_image_for_model(model_id: str) -> Path | None:
    """在 images_root 下递归查找包含 model_id 的文件名（不区分大小写）。"""
    lower_id = model_id.lower()
    # 先尝试精确匹配文件名（model_id.xxx）
    for ext in IMAGE_EXTS:
        p = images_root / f"{model_id}{ext}"
        if p.exists():
            return p
    # 再做一次递归检索，匹配包含 model_id 的文件
    for p in images_root.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMAGE_EXTS and lower_id in p.stem.lower():
            return p
    return None


def remove_background_to_rgba(img_path: Path) -> np.ndarray:
    """使用 rembg 去背景，返回 RGBA 的 numpy 数组。"""
    data = img_path.read_bytes()
    out_bytes = remove(data)  # rembg 返回 PNG 字节（通常含 alpha）
    arr = np.frombuffer(out_bytes, dtype=np.uint8)
    rgba = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
    if rgba is None:
        raise ValueError(f"解码去背景输出失败: {img_path}")
    # 确保为 4 通道
    if rgba.ndim == 3 and rgba.shape[2] == 3:
        alpha = np.full((rgba.shape[0], rgba.shape[1], 1), 255, dtype=rgba.dtype)
        rgba = np.concatenate([rgba, alpha], axis=2)
    return rgba


failed = []
processed = 0

for item in selection:
    model_id = item.get("model_id")
    if not model_id:
        continue
    img_path = find_image_for_model(model_id)
    if img_path is None:
        print(f"[缺失] 未找到图片: {model_id}")
        failed.append((model_id, "not_found"))
        continue
    try:
        rgba = remove_background_to_rgba(img_path)
        out_path = rgba_out_dir / f"{model_id}.png"
        # cv2.imwrite 需要 BGR(A)，rembg 输出已是 BGRA；直接保存
        ok = cv2.imwrite(str(out_path), rgba)
        if not ok:
            raise RuntimeError("cv2.imwrite 失败")
        processed += 1
        if processed % 10 == 0:
            print(f"已处理 {processed} 张...")
    except Exception as e:
        print(f"[失败] {model_id}: {e}")
        failed.append((model_id, str(e)))

print(f"完成。成功: {processed}，失败: {len(failed)}，输出目录: {rgba_out_dir}")
if failed:
    print("部分失败列举前 5 条:")
    for x in failed[:5]:
        print("  ", x)



完成。成功: 5，失败: 0，输出目录: /Users/apple/Desktop/VirtualFurnishing/outputs/rgba


In [10]:
from pathlib import Path
from typing import Tuple, List
import math
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

PROJECT_ROOT = Path("/Users/apple/Desktop/VirtualFurnishing")
images_root = PROJECT_ROOT / "data" / "modern_images"
rgba_out_dir = PROJECT_ROOT / "outputs" / "rgba"
compare_out_dir = PROJECT_ROOT / "outputs" / "compare"
compare_out_dir.mkdir(parents=True, exist_ok=True)

# 尝试加载系统字体（若失败回退到默认）
try:
    font = ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial Unicode.ttf", 18)
except Exception:
    font = ImageFont.load_default()


def to_pil(image_bgr_or_bgra: np.ndarray) -> Image.Image:
    if image_bgr_or_bgra.ndim == 2:
        return Image.fromarray(image_bgr_or_bgra)
    if image_bgr_or_bgra.shape[2] == 3:
        return Image.fromarray(cv2.cvtColor(image_bgr_or_bgra, cv2.COLOR_BGR2RGB))
    if image_bgr_or_bgra.shape[2] == 4:
        return Image.fromarray(cv2.cvtColor(image_bgr_or_bgra, cv2.COLOR_BGRA2RGBA))
    raise ValueError("Unsupported image shape")


def render_checkerboard(size: Tuple[int, int], cell: int = 16) -> Image.Image:
    w, h = size
    bg = Image.new("RGB", size, (220, 220, 220))
    draw = ImageDraw.Draw(bg)
    for y in range(0, h, cell):
        for x in range(0, w, cell):
            if (x // cell + y // cell) % 2 == 0:
                draw.rectangle([x, y, x + cell, y + cell], fill=(245, 245, 245))
    return bg


def alpha_composite_on_checker(rgba_arr: np.ndarray, max_side: int = 512) -> Image.Image:
    rgba = to_pil(rgba_arr)
    # 等比缩放到不超过 max_side
    ratio = min(max_side / rgba.width, max_side / rgba.height, 1.0)
    new_size = (max(1, int(rgba.width * ratio)), max(1, int(rgba.height * ratio)))
    rgba = rgba.resize(new_size, Image.LANCZOS)
    bg = render_checkerboard(rgba.size)
    bg = bg.convert("RGBA")
    bg.alpha_composite(rgba, (0, 0))
    return bg.convert("RGB")


def resize_max_side(img: Image.Image, max_side: int = 512) -> Image.Image:
    ratio = min(max_side / img.width, max_side / img.height, 1.0)
    return img.resize((max(1, int(img.width * ratio)), max(1, int(img.height * ratio))), Image.LANCZOS)


def make_side_by_side(original_path: Path, rgba_path: Path, title: str, max_side: int = 512) -> Image.Image:
    # 读原图
    orig_bgr = cv2.imread(str(original_path), cv2.IMREAD_COLOR)
    if orig_bgr is None:
        raise FileNotFoundError(f"原图无法读取: {original_path}")
    # 读抠图结果
    cut = cv2.imread(str(rgba_path), cv2.IMREAD_UNCHANGED)
    if cut is None:
        raise FileNotFoundError(f"抠图无法读取: {rgba_path}")

    orig_pil = resize_max_side(to_pil(orig_bgr), max_side)
    cut_vis = alpha_composite_on_checker(cut, max_side)

    padding = 16
    title_h = 32
    canvas_w = orig_pil.width + cut_vis.width + padding * 3
    canvas_h = max(orig_pil.height, cut_vis.height) + padding * 2 + title_h
    canvas = Image.new("RGB", (canvas_w, canvas_h), (255, 255, 255))
    draw = ImageDraw.Draw(canvas)

    # 标题
    draw.text((padding, padding), title, font=font, fill=(0, 0, 0))

    # 图片位置
    y0 = padding + title_h
    x1 = padding
    x2 = x1 + orig_pil.width + padding

    canvas.paste(orig_pil, (x1, y0))
    canvas.paste(cut_vis, (x2, y0))

    # 标注
    draw.text((x1, y0 - 24), "Original", font=font, fill=(80, 80, 80))
    draw.text((x2, y0 - 24), "Cutout", font=font, fill=(80, 80, 80))

    return canvas


def build_all_comparisons(items: List[dict], limit: int | None = None) -> list[Path]:
    out_paths: list[Path] = []
    count = 0
    for item in items:
        if limit is not None and count >= limit:
            break
        model_id = item.get("model_id")
        if not model_id:
            continue
        # 原图路径复用查找函数（已在上文定义）
        orig = find_image_for_model(model_id)
        cut = rgba_out_dir / f"{model_id}.png"
        if orig is None or not cut.exists():
            # 跳过缺失
            continue
        try:
            title = f"{model_id} | {item.get('category','')} | {item.get('style','')}"
            canvas = make_side_by_side(orig, cut, title)
            save_path = compare_out_dir / f"{model_id}_compare.png"
            canvas.save(save_path)
            out_paths.append(save_path)
            count += 1
        except Exception as e:
            print(f"对比生成失败 {model_id}: {e}")
    print(f"已生成对比图 {len(out_paths)} 张，输出目录: {compare_out_dir}")
    return out_paths


def grid_collage(image_paths: List[Path], cols: int = 2, cell_w: int = 640, cell_h: int = 480, gap: int = 16) -> Image.Image:
    if not image_paths:
        raise ValueError("无可用对比图构建拼图")
    rows = math.ceil(len(image_paths) / cols)
    W = cols * cell_w + (cols + 1) * gap
    H = rows * cell_h + (rows + 1) * gap
    board = Image.new("RGB", (W, H), (255, 255, 255))
    for idx, p in enumerate(image_paths):
        try:
            img = Image.open(p).convert("RGB")
        except Exception:
            continue
        img = resize_max_side(img, min(cell_w, cell_h))
        r = idx // cols
        c = idx % cols
        x = gap + c * (cell_w + gap) + (cell_w - img.width) // 2
        y = gap + r * (cell_h + gap) + (cell_h - img.height) // 2
        board.paste(img, (x, y))
    return board

# 运行：为所有 selection 生成对比 + 拼图
compare_list = build_all_comparisons(selection)
if compare_list:
    collage = grid_collage(compare_list, cols=2, cell_w=720, cell_h=540, gap=24)
    collage_path = PROJECT_ROOT / "outputs" / "selection_collage.png"
    collage.save(collage_path)
    print(f"拼图已保存: {collage_path}")
else:
    print("没有生成任何对比图（可能原图或抠图缺失）")



已生成对比图 5 张，输出目录: /Users/apple/Desktop/VirtualFurnishing/outputs/compare
拼图已保存: /Users/apple/Desktop/VirtualFurnishing/outputs/selection_collage.png
